In [9]:
import scanpy as sc
import Spectra as spc
import pandas as pd
from Spectra import Spectra_util as spc_tl
from Spectra import K_est as kst
from Spectra import default_gene_sets

In [3]:
annotations = spc.default_gene_sets.load()
annotations["global"]["all_wnt-beta-catenin-signaling"]

['HDAC2',
 'LEF1',
 'CUL1',
 'TP53',
 'GNAI1',
 'DLL1',
 'SKP2',
 'AXIN1',
 'KAT2A',
 'CSNK1E',
 'MAML1',
 'FZD8',
 'NOTCH1',
 'MYC',
 'RBPJ',
 'JAG2',
 'PTCH1',
 'PPARD',
 'HDAC5',
 'AXIN2',
 'NUMB',
 'PSEN2',
 'NKD1',
 'WNT1',
 'TCF7',
 'FRAT1',
 'HDAC11',
 'ADAM17',
 'CCND2',
 'HEY1',
 'NCSTN',
 'JAG1',
 'DVL2',
 'NOTCH4',
 'NCOR2',
 'TPTEP2-CSNK1E',
 'FZD1',
 'CTNNB1',
 'DKK4',
 'HEY2',
 'WNT6',
 'DKK1',
 'WNT5B']

In [17]:
len(annotations)

15

In [ ]:
#TODO: check how they got this dataset + send info to see if its appropriate for our data

In [4]:
# Load your data
adata = sc.read_h5ad('patient_data/anal_pc5_c21_S1.filtered.h5ad')

check if gene names in h5ed file match the annotastions

In [10]:
adata_df = pd.DataFrame(
    adata.X.toarray() if hasattr(adata.X, 'toarray') else adata.X,
    index=adata.obs_names,
    columns=adata.var_names
)
adata_df.head()

,SAMD11,NOC2L,KLHL17,PLEKHN1,PERM1,HES4,ISG15,AGRN,RNF223,C1orf159,...,TTLL8,CT83,MAGEA10,S100A7L2,KRT39,NXNL1,DEFB132,GAB4,PPP1R2C,SPANXD
S1_AAACAAGCAACAGCACACTTTAGG_anal_34449_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
S1_AAACAAGCAATTGAGTACTTTAGG_anal_34449_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
S1_AAACAAGCACGTAAATACTTTAGG_anal_34449_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
S1_AAACAAGCACTATCACACTTTAGG_anal_34449_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
S1_AAACAAGCAGAATGAAACTTTAGG_anal_34449_1_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
adata_gene_list = adata_df.columns.tolist()
len(adata_gene_list)

17781

In [16]:
annot_set = set(annotations)
s1_set = set(adata_df)

# Calculate intersections and differences
in_both = annot_set & s1_set
only_in_list1 = annot_set - s1_set
only_in_list2 = s1_set - annot_set

# Print results
print(f"In both lists: {len(in_both)} items")
print(f"Only in Annotation: {len(only_in_list1)} items")
print(f"Only in S1 gene list: {len(only_in_list2)} items")

In both lists: 0 items
Only in Annotation: 15 items
Only in S1 gene list: 17781 items


spectra model

In [ ]:
model = spc.est_spectra(
    adata=adata, 
    gene_set_dictionary=annotations,
    use_highly_variable=True,
    cell_type_key='cell_type_annotations',  # Your obs column name
    use_weights=True,
    lam=0.1,
    delta=0.001,
    kappa=0.00001,
    rho=0.00001,
    use_cell_types=True,
    n_top_vals=25,
    label_factors=True,
    overlap_threshold=0.2,
    num_epochs=10000  # They recommend 10,000
)